In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph()
graph

In [ ]:
movie_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [ ]:
movie_query

In [ ]:
graph.query(movie_query)

In [ ]:
graph.refresh_schema()

In [ ]:
print(graph.schema)

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama3-8b-8192")

In [ ]:
from langchain.chains import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
chain

In [ ]:
response = chain.invoke({"query":"Who was the director of the movie Casino"})
response

In [ ]:
response=chain.invoke({"query": "Give me the entire cast of Casino"})
response

In [ ]:
response=chain.invoke({"query": "Give me release year of Jumanji"})
response

In [ ]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "Match (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino",
        "query": "MATCH (m:Movie {title:'Casino'})<- [:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name:'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {title:'Schindler\'s List'})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which directors have made movies with at least three diferent actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]-> (m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:ACTOR)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
    
]